# FAANG Stock Analysis and Predictor

In this notebook, we are taking a look at the five most popular and best performing American technology companies: Facebook, Amazon, Apple, Netflix, and Google. In the notebook, there will be an analysis on the stock performance in the year 2020, as well as a prediction algorithm (using machine learning) that will be run on each of the five stocks to ulimately provide a complete analysis of all five stocks.

### Table Of Contents

* [Setup](#setup)


## Setup <a class="anchor" id="setup"></a>

In [131]:
import math
from datetime import date, timedelta
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [132]:
# Get the data from all of the FAANG stocks
five_years_ago = date.today() - timedelta(days=5*365)
fb_data = web.DataReader('FB', data_source='yahoo', start=five_years_ago, end=date.today())
amzn_data = web.DataReader('AMZN', data_source='yahoo', start=five_years_ago, end=date.today())
aapl_data = web.DataReader('AAPL', data_source='yahoo', start=five_years_ago, end=date.today())
nflx_data = web.DataReader('NFLX', data_source='yahoo', start=five_years_ago, end=date.today())
goog_data = web.DataReader('GOOG', data_source='yahoo', start=five_years_ago, end=date.today())

In [133]:
# Put all of the adj. closing prices into one dataframe
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOG']
faang_data = pd.DataFrame(data=list, index=fb_data.index, columns=tickers)
faang_data['FB'] = fb_data['Adj Close']
faang_data['AMZN'] = amzn_data['Adj Close']
faang_data['AAPL'] = aapl_data['Adj Close']
faang_data['NFLX'] = nflx_data['Adj Close']
faang_data['GOOG'] = goog_data['Adj Close']

In [134]:
# Show the data
faang_data

,FB,AMZN,AAPL,NFLX,GOOG
Date,,,,,
2015-10-26,103.769997,608.609985,26.632969,103.040001,712.780029
2015-10-27,103.699997,611.010010,26.464319,103.070000,708.489990
2015-10-28,104.199997,617.099976,27.554773,105.800003,712.950012
2015-10-29,104.879997,626.549988,27.845867,105.120003,716.919983
2015-10-30,101.970001,625.900024,27.607912,108.379997,710.809998
...,...,...,...,...,...
2020-10-16,265.929993,3272.709961,119.019997,530.789978,1573.010010
2020-10-19,261.399994,3207.209961,115.980003,530.719971,1534.609985
2020-10-20,267.559998,3217.010010,117.510002,525.419983,1555.930054


## Technical Indicators Analysis
1. Simple Moving Averages (SMA)
2. Exponential Moving Averages (EMA)
3. Relative Strength Index (RSI)
4. Moving Average Convergence/Divergence (MACD)
5. Stochastic Oscillator
6. Bollinger Bands


In [135]:
# Function to get the SMA

def get_SMA(ticker, period):
    SMA = pd.DataFrame()
    SMA['SMA'] = faang_data[ticker].rolling(window=period).mean()
    return SMA

In [136]:
# Function to get the EMA

def get_EMA(ticker, period):
    EMA = pd.DataFrame()
    EMA['EMA'] = faang_data[ticker].ewm(span=period, adjust=False).mean()
    
    return EMA

In [137]:
# Function to get the RSI

def get_RSI(ticker):
    RSI = pd.DataFrame()
    RSI['Closing Prices'] = faang_data[ticker]
    RSI['Change'] = RSI['Closing Prices'].diff()
    
    changes = RSI['Change'].tolist()
    
    gains = []
    losses = []
    
    for change in changes:
        if change > 0:
            gains.append(change)
            losses.append(0)
        else:
            gains.append(0)
            losses.append(-change)
    
    gains =  pd.Series(gains)
    losses = pd.Series(losses)
    
    RSI['Gains'] = gains.values
    RSI['Losses'] = losses.values
    
    RSI['Avg. Gain'] = RSI['Gains'].rolling(window=14).mean()
    RSI['Avg. Loss'] = RSI['Losses'].rolling(window=14).mean()
    
    RSI['RS'] = RSI['Avg. Gain'] / RSI['Avg. Loss']
    
    RSI['RSI'] = 100 - 100 / (1 + RSI['RS'])
    
    
    return RSI

In [138]:
# Function to get the MACD crossover

def get_MACD(ticker):
    ShortEMA = get_EMA(ticker, 12)
    LongEMA = get_EMA(ticker, 26)
    
    MACD = pd.DataFrame()
    MACD['12 Day EMA'] = ShortEMA['EMA']
    MACD['26 Day EMA'] = LongEMA['EMA']
    MACD['MACD'] = MACD['12 Day EMA'] - MACD['26 Day EMA']
    
    return MACD

In [139]:
# Function to get the Stochastic Oscillator

def get_stochastic_oscillator(ticker):
    stoch_osc = pd.DataFrame()
    
    stoch_osc['Closing Price'] = faang_data[ticker]
    stoch_osc['14 Day Low'] = stoch_osc['Closing Price'].rolling(window=14).min()
    stoch_osc['14 Day High'] = stoch_osc['Closing Price'].rolling(window=14).max()
    
    stoch_osc['Stochastic Oscillator'] = (stoch_osc['Closing Price'] - stoch_osc['14 Day Low'])
    stoch_osc['Stochastic Oscillator'] = stoch_osc['Stochastic Oscillator'] / (stoch_osc['14 Day High'] - stoch_osc['14 Day Low']) * 100
    
    
    
    return stoch_osc

In [140]:
# Function to get the Bollinger Bands

def get_bollinger_bands(ticker):
    period = 20  
    bollinger_bands = get_SMA(ticker, period)
    
    bollinger_bands['Std. Deviation'] = faang_data[ticker].rolling(window=period).std()
    
    bollinger_bands['Upper Band'] = bollinger_bands['SMA'] + (bollinger_bands['Std. Deviation'] * 2)
    bollinger_bands['Lower Band'] = bollinger_bands['SMA'] - (bollinger_bands['Std. Deviation'] * 2)
    
    
    
    return bollinger_bands

In [144]:
# Function to get the MFI

def get_MFI(data):
    MFI = pd.DataFrame()
    
    MFI['High'] = data['High']
    MFI['Close'] = data['Adj Close']
    MFI['Low'] = data['Low']
    
    MFI['Typical Price'] = (MFI['High'] + MFI['Close'] + MFI['Low']) / 3
    typ_price = MFI['Typical Price'].tolist()
    
    period = 14
    MFI['Raw MF'] = MFI['Typical Price'] * data['Volume']
    
    money_flow = MFI['Raw MF'].tolist()
    
    
    pos_mf = []
    neg_mf = []
    
    for i in range(len(money_flow)):
        if typ_price[i] > typ_price[i - 1]:
            pos_mf.append(money_flow[i])
            neg_mf.append(0)
        elif typ_price[i] < typ_price[i - 1]:
            pos_mf.append(0)
            neg_mf.append(money_flow[i])
        else:
            pos_mf.append(0)
            neg_mf.append(0)
    
    MFI['Positive MF'] = pd.Series(pos_mf).values
    MFI['Negative MF'] = pd.Series(neg_mf).values
    
    MFI['MF Ratio'] = MFI['Positive MF'].rolling(window=14).sum() / MFI['Negative MF'].rolling(window=14).sum()
    MFI['MFI'] = 100 - 100 / (1 + MFI['MF Ratio'])
    
    return MFI

In [160]:
# Function to get the OBV

def get_OBV(data):
    OBV = pd.DataFrame()
    
    OBV['Volume'] = data['Volume']
    OBV['Close'] = data['Adj Close']
    
    volume = OBV['Volume'].tolist()
    closing_prices = OBV['Close'].tolist()
    
    current_OBV = []
    current_OBV.append(0)
    
    for i in range(1, len(closing_prices)):
        if closing_prices[i] > closing_prices[i-1]:
            current_OBV.append(current_OBV[i-1] + volume[i])
        elif closing_prices[i] < closing_prices[i-1]:
            current_OBV.append(current_OBV[i-1] - volume[i])
        else:
            current_OBV.append(current_OBV[i-1])
    
    
    OBV['OBV'] = pd.Series(current_OBV).values
    
    return OBV

In [179]:
# Function to get A/D Indicator

def get_AD_indicator(data):
    AD = pd.DataFrame()
    
    AD['High'] = data['High']
    AD['Low'] = data['Low']
    AD['Close'] = data['Adj Close']
    AD['Volume'] = data['Volume']
    AD['CLV'] = (AD['Close'] - AD['Low']) - (AD['High'] - AD['Close'])
    AD['CLV'] = AD['CLV'] / (AD['High'] - AD['Low'])
    AD['CMFV'] = AD['CLV'] * AD['Volume']
    
    current_ad = []
    current_cmfv = AD['CMFV'].tolist()
    
    current_ad.append(current_cmfv[0])
    
    for i in range(1, len(current_cmfv)):
        current_ad.append(current_ad[i-1] + current_cmfv[i])
    
    AD['AD'] = pd.Series(current_ad).values
    
    
    return AD

In [263]:
# Function to get ADX

def get_adx(data):
    ADX = pd.DataFrame()
    
    ADX['High'] = data['High']
    ADX['Low'] = data['Low']
    ADX['Close'] = data['Adj Close']
    ADX['H - L'] = ADX['High'] - ADX['Close']
    ADX['|H - Cp|'] = (ADX['High'] - ADX['Close'].shift()).abs()
    ADX['|L - Cp|'] = (ADX['Low'] - ADX['Close'].shift()).abs()
    
    ADX['TR'] = ADX[['H - L', '|H - Cp|', '|L - Cp|']].max(axis=1)
    
    
    pos_dm1 = (ADX['High'] - ADX['High'].shift()).tolist()
    neg_dm1 = ((ADX['Low'] - ADX['Low'].shift()).abs()).tolist()
    
    
    for i in range(1, len(pos_dm1)):
        if pos_dm1[i] > neg_dm1[i]:
            neg_dm1[i] = 0
        else:
            pos_dm1[i] = 0
    
    
    ADX['+DM1'] = pd.Series(pos_dm1).values
    ADX['-DM1'] = pd.Series(neg_dm1).values
    
    ADX['TR14'] = ADX['TR'].rolling(window=14).sum()
    ADX['TR14'] = ADX['TR14'] - (ADX['TR14'] / 14) + ADX['TR']
    
    ADX['+DM14'] = ADX['+DM1'].rolling(window=14).sum()
    ADX['+DM14'] = ADX['+DM14'] - (ADX['+DM14'] / 14) + ADX['+DM1']
    
    ADX['-DM14'] = ADX['-DM1'].rolling(window=14).sum()
    ADX['-DM14'] = ADX['-DM14'] - (ADX['-DM14'] / 14) + ADX['-DM1']
    
    ADX['+DI14'] = (ADX['+DM14'] / ADX['TR14']) * 100
    ADX['-DI14'] = (ADX['-DM14'] / ADX['TR14']) * 100
    
    ADX['DX'] = (((ADX['+DI14'] - ADX['-DI14']).abs()) / (ADX['+DI14'] + ADX['-DI14'])) * 100
    ADX['ADX'] = ADX['DX'].rolling(window=14).mean()
    ADX['ADX'] = ((ADX['ADX'] * 13) + ADX['DX']) / 14
    
    return ADX

In [536]:
def get_aroon_osc(data):
    aroon_osc = pd.DataFrame()
    
    period = 25
    
    aroon_osc['High'] = data['High']
    aroon_osc['Low'] = data['Low']
    aroon_osc['25 Day High'] = aroon_osc['High'].rolling(window=25).max()
    aroon_osc['25 Day Low'] = aroon_osc['Low'].rolling(window=25).max()
    
    
    
    
    return aroon_osc

In [537]:
aroon_osc = get_aroon_osc(fb_data)
aroon_osc.head(60)

,High,Low,25 Day High,25 Day Low
Date,,,,
2015-10-26,103.809998,101.550003,NaN,NaN
2015-10-27,104.099998,102.400002,NaN,NaN
2015-10-28,104.480003,102.919998,NaN,NaN
2015-10-29,105.120003,103.519997,NaN,NaN
2015-10-30,104.720001,101.650002,NaN,NaN
2015-11-02,103.470001,101.180000,NaN,NaN
2015-11-03,103.650002,102.400002,NaN,NaN
2015-11-04,104.139999,102.480003,NaN,NaN
2015-11-05,110.650002,107.949997,NaN,NaN
